In [1]:
import cv2 
from matplotlib import pyplot as plt
import numpy as np 

In [ ]:
import paho.mqtt.client as mqtt
import cv2
import csv
from datetime import datetime
import threading

# MQTT ayarları
MQTT_BROKER = "192.168.113.135"  # broker ' ı kurduğumuz ip adresi'
MQTT_PORT = 1883
MQTT_TOPIC = "esp32/alici"

csv_filename = datetime.now().strftime('%Y%m%d_%H%M%S') + '_imu_data.csv'


# kamera ayarlaraı
cap = cv2.VideoCapture(0) # bilgisayara bağlı hangi kameradan veri alınması isteniyorsa orayı seçiyoruz.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = 20.0

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_filename = datetime.now().strftime('%Y%m%d_%H%M%S') + '.avi'
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

x_data, y_data, z_data = [], [], []
gx_data, gy_data, gz_data = [], [], []

# q ya bastığında ver almasını 0'lamak için
stop_flag = False

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])

def on_connect(client, userdata, flags, rc):
    print(f"Bağlandı: {rc}")
    client.subscribe(MQTT_TOPIC)

def on_message(client, userdata, msg):
    global x_data, y_data, z_data, gx_data, gy_data, gz_data, stop_flag
    if stop_flag:
        return
    message = msg.payload.decode()
    print(f"Gelen mesaj: {message}")
    data = message.split(" | ")
    
    # ivme verileri
    ax = float(data[0].split(": ")[1])
    ay = float(data[1].split(": ")[1])
    az = float(data[2].split(": ")[1])
    x_data.append(ax)
    y_data.append(ay)
    z_data.append(az)
    
    #jiroskop verileri
    gx = float(data[3].split(": ")[1])
    gy = float(data[4].split(": ")[1])
    gz = float(data[5].split(": ")[1])
    gx_data.append(gx)
    gy_data.append(gy)
    gz_data.append(gz)
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, ax, ay, az, gx, gy, gz])

def imu_thread():
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.loop_forever()

def video_thread():
    global stop_flag
    while cap.isOpened():
        ret, frame = cap.read()  
        if not ret:
            break  
        
        
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cv2.putText(frame, timestamp, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        out.write(frame)
        
        cv2.imshow('webcam', frame)
        
        # q tuşuna basıldığında kamera verisi ve imu verisi dursun
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_flag = True
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# İki ayrı işi aynı yapmak için .thread kullanılır.
imu_thread = threading.Thread(target=imu_thread)
video_thread = threading.Thread(target=video_thread)

imu_thread.start()
video_thread.start()

imu_thread.join()
video_thread.join()
